# **DETECTOR DE OBJETOS CON YOLO V11**

In [1]:
# INSTALACIÓN DE LIBRERÍAS NECESARIASA
#pip install ultralytics
#pip install roboflow

In [2]:
# carga de utils
import sys
sys.path.append('../utils')
from utils import generar_lineas_por_manzana, procesar_jsons_en_carpeta, tabulate_jsons_from_folder, copiar_archivos_seleccionados

d:\Github\Block-Detection\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# carga de librerías
from ultralytics import YOLO
import pandas as pd
import numpy as np
import shutil

In [4]:
# Procesamiento de la data de los labels
procesar_jsons_en_carpeta('../data/raw/dataset_vertices_256/metadata_y_256/','../data/interim_yolo/dataset_labels/')

In [5]:
# Definición de train y test
jsons = tabulate_jsons_from_folder('../data/raw/data_set_256/metadata_y_256/')

l1 = jsons[jsons['num_polygons_in_window']>=1]['id']
l0 = jsons[jsons['num_polygons_in_window']==0]['id'].sample(n=int(np.round(len(l1)/5)), random_state=42)
l = pd.concat([l1,l0])

l_train = l.sample(frac=0.8, random_state=42)
l_test = l.drop(l_train.index)

In [9]:
# Adecuación al directorio requerido por Yolo
copiar_archivos_seleccionados('../data/raw/data_set_256/dataset_x_256/', '../data/interim_yolo/train/images/', [str(x)+'.png' for x in l_train])
copiar_archivos_seleccionados('../data/raw/data_set_256/dataset_x_256/', '../data/interim_yolo/valid/images/', [str(x)+'.png' for x in l_test])

copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/train/labels/', [str(x)+'.txt' for x in l_train])
copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/valid/labels/', [str(x)+'.txt' for x in l_test])

In [ ]:
# DESCARGA DE DATASET PERSONALIZADO DESDE ROBOFLOW
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="YP6UX0p6bmRnwAjcSbfY")
#project = rf.workspace("nella-hyakz").project("custom_object_detector_yolo11_v2-rotnm")
#version = project.version(1)
#dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...


In [7]:
# CARGA DEL MODELO BASE YOLO V11
from ultralytics import YOLO
model = YOLO("yolo11s.pt")

In [ ]:
# ENTRENAMIENTO DEL MODELO PERSONALIZADO
data_path = "custom_object_detector_yolo11_v2-1/data.yaml"
results= model.train (data=data_path,
epochs=15,
imgsz=640)

Ultralytics 8.3.227  Python-3.10.6 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom_object_detector_yolo11_v2-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspe

# Predicciones

In [10]:
# CARGAR EL MODELO YA ENTRENADO
custom_model = YOLO("runs/detect/train/weights/best.pt")

In [11]:
# REALIZAR PREDICCIONES SOBRE ALGUNAS IMÁGENES
res = custom_model("custom_object_detector_yolo11_v2-1/test/images")


image 1/1 e:\INEI\PI\MODELO_YOLO_V11\custom_object_detector_yolo11_v2-1\test\images\prueba.jpg: 384x640 11 manzanass, 24.3ms
Speed: 4.5ms preprocess, 24.3ms inference, 15.3ms postprocess per image at shape (1, 3, 384, 640)


In [13]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[0].show()

In [6]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[1].show()

IndexError: list index out of range